In [4]:
!pip install selenium

In [5]:
# 키워드 검색
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/'+word
    return url

In [6]:
# 첫 번째 게시물 클릭
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
#select_first(driver)

In [7]:
# 게시물 정보 가져오기
# 본문 내용, 작성 일시, 위치 정보

import re
from bs4 import BeautifulSoup
import unicodedata

def get_content(driver):
    # html 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK>span')[0].text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ''
    
    # 본문의 해시태그 가져오기
    tags= re.findall(r'#[^\s,\\]+', content)
    
    # 작성 일자 정보 가져오기
    date = soup.select('time.FH9sR.Nzb55')[0]['title']
    
    # 위치 정보 가져오기
    try:
        place = soup.select('div.JF9hh')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
        
    # 수집한 정보 저장
    data = [content, date, place, tags]
    
    return data

#get_content(driver)

In [8]:
# 다음 게시물 열기
def move_next(driver):
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
#move_next(driver)

In [53]:
# 인스타그램 로그인
def instagram_login(id,pw):
    input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
    input_id.clear()
    input_id.send_keys(id)

    input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
    input_pw.clear()
    input_pw.send_keys(pw)
    input_pw.submit()
    time.sleep(3)
    
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()   #로그인정보 저장 '나중에 하기' 버튼 클릭
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click() #알림 설정 '나중에 하기' 버튼 클릭

In [119]:
# 게시물 정보 가져오기
# 닉네임, 작성 일시, 위치 정보, 해시태그

import re
from bs4 import BeautifulSoup
import unicodedata

def get_data(driver, n):
    # html 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK>span')[0].text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ''
    
    # 본문의 해시태그 가져오기
    tags= re.findall(r'#[^\s,\\]+', content)

    
    # 작성 일자 정보 가져오기
    date = soup.select('time.FH9sR.Nzb55')[0]['title']
    
    # 위치 정보 가져오기
    try:
        place = soup.select('div.JF9hh')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
        
    # 이름(사용자 이름) 가져오기
    name = driver.find_element_by_css_selector(".sqdOP.yWX7d._8A5w5.ZIAjV").text
    
    # 사용자 사진 가져오기
    userImgUrl = soup.select('div.Jv7Aj.mArmR.pZp3x')[n].img['src']
    print(userImgUrl)
    
    # 사진 가져오기
    imgUrl = soup.select('div.KL4Bh')[n].img['src']
    #print(imgUrl)
    
        
    # 수집한 정보 저장
    data = [imgUrl, name, userImgUrl, date, place, tags]
    
    return data

#get_data(driver, 1)

In [120]:
# 2. 키워드 검색
key_word = '원데이클래스'
url = insta_searching(key_word)

# 3. 검색 페이지 접속
driver.get(url)
time.sleep(4)

# 4. 첫 번째 게시글 열기
select_first(driver)

In [121]:
# 5. 여러 게시물 수집하기
results =[]

target = 5  # 크롤링할 게시글 수
for i in range(target):
    try:
        data = get_data(driver, i)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
        
print(results)

https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/24127150_146214572807336_1879551480562712576_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=yxhA26mMRQYAX8SMEyI&edm=AP_V10EBAAAA&ccb=7-4&oh=6d91fbb00d6aa529a0ae6f245674a261&oe=611889C5&_nc_sid=4f375e
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/191204484_499943227916304_7848707498602884047_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=ymGWqv8ONyAAX89Heez&edm=AP_V10EBAAAA&ccb=7-4&oh=3d35b1a026cd4ba218eaa97bfa16f738&oe=6117F6EF&_nc_sid=4f375e
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/157267203_127317015980071_3131037788886948735_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=XMprA9fptEcAX_4Alf7&edm=AP_V10EBAAAA&ccb=7-4&oh=00ef0f1a5c78ef1b4803f0eea4d4fd6b&oe=6118E73D&_nc_sid=4f375e
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/132005602_1647893762068942_4854535290532724068_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=9XuOm3ev2LAAX-bbqJu&

In [115]:
from PIL import Image
import requests

for i in range(target):
    imgUrl = results[i][0]
    username = results[i][1]
    print(i,": ",username,imgUrl)

    im = Image.open(urlopen(imgUrl))
    im

0 :  mingmiraclegreen https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/233564092_166647708789273_2773787315642028626_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=109&_nc_ohc=wCc4AIHhcUwAX-zfIWj&edm=AP_V10EBAAAA&ccb=7-4&oh=f67ef4db09045529def36bcc3f4c15e0&oe=611926B8&_nc_sid=4f375e
1 :  ballet_jini https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/233684886_2607636926208982_5287449811268705697_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=110&_nc_ohc=qBWW_IeN1hwAX_DqO_H&edm=APU89FABAAAA&ccb=7-4&oh=4534c221009bed77839125cb4bd151f4&oe=61174B26&_nc_sid=86f79a
2 :  i_cd_i https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/233679785_4202144759900090_8464782836325507763_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=FrqpQ3bT2tEAX9ECHTJ&edm=AP_V10EBAAAA&ccb=7-4&oh=9c8e78263c62d54f7fc67112fd302ce5&oe=61180230&_nc_sid=4f375e
3 :  on_the1f https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080

In [117]:
from urllib.request import urlopen

for i in range(target):
    imgUrl = results[i][0]
    print(i,": ",imgUrl)
    with urlopen(imgUrl) as f:
        with open('./img/' + results[i][1] + str(i) + '.jpg', 'wb') as h:
            img = f.read()
            h.write(img)

0 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/233564092_166647708789273_2773787315642028626_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=109&_nc_ohc=wCc4AIHhcUwAX-zfIWj&edm=AP_V10EBAAAA&ccb=7-4&oh=f67ef4db09045529def36bcc3f4c15e0&oe=611926B8&_nc_sid=4f375e
1 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/233684886_2607636926208982_5287449811268705697_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=110&_nc_ohc=qBWW_IeN1hwAX_DqO_H&edm=APU89FABAAAA&ccb=7-4&oh=4534c221009bed77839125cb4bd151f4&oe=61174B26&_nc_sid=86f79a
2 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/233679785_4202144759900090_8464782836325507763_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=FrqpQ3bT2tEAX9ECHTJ&edm=AP_V10EBAAAA&ccb=7-4&oh=9c8e78263c62d54f7fc67112fd302ce5&oe=61180230&_nc_sid=4f375e
3 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/234136918_929415177611017_349565021456132975

In [122]:
for i in range(target):
    userImgUrl = results[i][2]
    print(i,": ",userImgUrl)
    with urlopen(userImgUrl) as f:
        with open('./user/' + 'user_'+results[i][1] + str(i) + '.jpg', 'wb') as h:
            img = f.read()
            h.write(img)

0 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/24127150_146214572807336_1879551480562712576_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=yxhA26mMRQYAX8SMEyI&edm=AP_V10EBAAAA&ccb=7-4&oh=6d91fbb00d6aa529a0ae6f245674a261&oe=611889C5&_nc_sid=4f375e
1 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/191204484_499943227916304_7848707498602884047_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=ymGWqv8ONyAAX89Heez&edm=AP_V10EBAAAA&ccb=7-4&oh=3d35b1a026cd4ba218eaa97bfa16f738&oe=6117F6EF&_nc_sid=4f375e
2 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/157267203_127317015980071_3131037788886948735_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=XMprA9fptEcAX_4Alf7&edm=AP_V10EBAAAA&ccb=7-4&oh=00ef0f1a5c78ef1b4803f0eea4d4fd6b&oe=6118E73D&_nc_sid=4f375e
3 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/132005602_1647893762068942_4854535290532724068_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=

In [41]:
# 여러 게시글 정보 수집하기

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 1. 크롬 브라우저 열기 및 로그인
driver = webdriver.Chrome('c:/users/mindo/like_lion/oneday/chromedriver.exe')
site = 'https://www.instagram.com/'
driver.get(site)
time.sleep(2)

email = 'mindori1019@gmail.com'
password = 'alswn4249!'
instagram_login(email, password)

# 2. 키워드 검색
key_word = '원데이클래스'
url = insta_searching(key_word)

# 3. 검색 페이지 접속
driver.get(url)
time.sleep(4)

# 4. 첫 번째 게시글 열기
select_first(driver)

# 5. 여러 게시물 수집하기
results =[]

target = 5  # 크롤링할 게시글 수
for i in range(target):
    try:
        data = get_data(driver, i)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)


WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.131)


In [12]:
# 엑셀에 데이터 저장
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns =  ['photo','name', 'userphoto', 'date', 'place', 'tags']
results_df.to_excel('c:/users/mindo/like_lion/crawling_instagram.xlsx', index = False)